## Demonstration of the User-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing user interactions, such as scroll length and read time, to identify users with similar behavior. 
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.



In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.collaborative.user_based_CF import UserBasedCollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data Import and EDA

In [2]:
data_reader = ParquetDataReader()
articles_df = data_reader.read_data('../../data/articles.parquet')
train_behaviors_df = data_reader.read_data('../../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../../data/train/history.parquet')
document_vectors_df = data_reader.read_data('../../data/document_vector.parquet')

We check the size of the data. From the sizes we learn that:
<ol>
  <li>We have 20738 unique articles</li>
  <li>We have 15143 unique users</li>
  <li>We have 232887 interactions in the testset</li>
</ol> 

In [3]:
print("articles_df has the size:         ", articles_df.shape)
print("train_behaviors_df has the size:  ", train_behaviors_df.shape)
print("train_history_df has the size:    ", train_history_df.shape)
print("document_vectors_df has the size: ", document_vectors_df.shape)

articles_df has the size:          (20738, 21)
train_behaviors_df has the size:   (232887, 17)
train_history_df has the size:     (15143, 5)
document_vectors_df has the size:  (125541, 2)


### Validation Set

In [4]:
test_behaviours_df = data_reader.read_data('../../data/validation/behaviors.parquet')
test_behaviours_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [5]:
# Combine train and test behaviors
combined_df = pl.concat([train_behaviors_df, test_behaviours_df])

# Generate a random mask for splitting
n = combined_df.height  # Total number of rows
test_mask = np.random.rand(n) < 0.30  # 30% test, 70% train

# Apply the mask
test_behaviors_df = combined_df.filter(test_mask)
train_behaviors_df = combined_df.filter(~test_mask)

# Verify the split
print(f"Train size: {train_behaviors_df.shape[0]}, Test size: {test_behaviors_df.shape[0]}")

Train size: 334815, Test size: 142719


### Table Contents

The information on news articles. As we are going to perform user-user CF, this table is not neccesary

In [6]:
articles_df.head()

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun fyret i Sønderjys…","""FODBOLD: Morten Bruun fyret me…",2023-06-29 06:20:39,false,"""Kemien mellem spillerne i Supe…",2006-05-01 14:28:40,[3177953],"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytter på landet""","""I landets tyndest befolkede om…",2023-06-29 06:20:43,false,"""Det frække erhverv rykker på l…",2007-03-24 08:27:59,[3184029],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvornår er man utro?""","""En flirtende sms til den flott…",2023-06-29 06:20:46,false,"""De fleste af os mener, at et t…",2007-01-18 10:30:37,[3030463],"""article_default""","""https://ekstrabladet.dk/sex_og…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


Each file consists of seven days of impression logs. The train_behaviors_df table contains all information about interactions between users and items, and can be used as a basis for user-user CF. <strong>Therefore we only need this table</strong>.

In [7]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0
153078,9777492,2023-05-24 07:13:46,7.0,100.0,1,"[9778021, 9778627, … 7213923]",[9778226],151570,false,null,null,null,false,1976,4.0,21.0


Each file consists of users' click histories collected over 21 days period. This table does contain the same values as the train_behaviours_df, but as that table is easier to work with we will use train_behaviours_df over this one

In [8]:
train_history_df.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"


List of vectors for each article. This is used to describe the items. It could be used for item-item CF, but is not relevant to user-user CF.  <strong>This table will therefore not be used</strong>

In [9]:
document_vectors_df.head()

article_id,document_vector
i32,list[f32]
3000022,"[0.065424, -0.047425, … 0.035706]"
3000063,"[0.028815, -0.000166, … 0.027167]"
3000613,"[0.037971, 0.033923, … 0.063961]"
3000700,"[0.046524, 0.002913, … 0.023423]"
3000840,"[0.014737, 0.024068, … 0.045991]"


From the analasys we see that we only need train_behaviour_df to perform user-user CF

## Preprocessing

### Remove Non-Needed Values

We see that we have several items that are not required for performing user-user CF

In [10]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0
153078,9777492,2023-05-24 07:13:46,7.0,100.0,1,"[9778021, 9778627, … 7213923]",[9778226],151570,false,null,null,null,false,1976,4.0,21.0


All information that does not describe a user, or a user-item interaction can therefore be removed

In [11]:
irelevant_columns = ["impression_time", "device_type", "article_ids_inview", "article_ids_clicked", "session_id", "next_read_time", "next_scroll_percentage"]
train_behaviors_df = train_behaviors_df.drop(irelevant_columns)
train_behaviors_df.head()

impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,i32,f32,f32,u32,bool,i8,i8,i8,bool
149474,null,13.0,null,139836,false,null,null,null,false
153068,9778682,78.0,100.0,151570,false,null,null,null,false
153070,9777492,26.0,100.0,151570,false,null,null,null,false
153075,9777492,26.0,100.0,151570,false,null,null,null,false
153078,9777492,7.0,100.0,151570,false,null,null,null,false


The remaining items are the ones that can be used. But already here we see that we have several features with lacking information. We should therefore treat this

### Account for Missing Values

We see here that alot of the behaviours contain missing values. The therefore have to either remove or replace the values

In [12]:
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(334815, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,235313,0,236821,0,0,311891,328335,325948,0


In [13]:
train_behaviors_df = train_behaviors_df.filter(train_behaviors_df["article_id"].is_not_null())
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99502, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,2570,0,0,92676,97239,96087,0


We see that of 70421, are there between 65-68000 missing values for gender, postcode and age. We therefore remove these as there is no use subsidizing them

In [14]:
train_behaviors_df = train_behaviors_df.drop(["gender", "postcode", "age"])
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99502, 7)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,is_subscriber
u32,u32,u32,u32,u32,u32,u32
0,0,0,2570,0,0,0


We still see that 2570/98967 rows are missing a scroll percentage. As this is very low (<3%) we can easily replace this. Intitially we just set scroll to 0

In [15]:
train_behaviors_df = train_behaviors_df.fill_null(strategy="zero")

### Account for Multiple Instances of the Same Article and User

By checking rows where the user_id and article_id are the same we see that we have 9855 instances where the user has read the same article multiple times

In [16]:
duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)

print(duplicates)

shape: (12_432, 3)
┌────────────┬─────────┬───────┐
│ article_id ┆ user_id ┆ count │
│ ---        ┆ ---     ┆ ---   │
│ i32        ┆ u32     ┆ u32   │
╞════════════╪═════════╪═══════╡
│ 9780925    ┆ 49157   ┆ 2     │
│ 9783965    ┆ 653390  ┆ 2     │
│ 9779204    ┆ 2096154 ┆ 2     │
│ 9784856    ┆ 218401  ┆ 4     │
│ 9778198    ┆ 348455  ┆ 6     │
│ …          ┆ …       ┆ …     │
│ 9780096    ┆ 690100  ┆ 2     │
│ 9785790    ┆ 184630  ┆ 2     │
│ 9789837    ┆ 1748430 ┆ 3     │
│ 9772963    ┆ 147171  ┆ 4     │
│ 9778628    ┆ 524998  ┆ 2     │
└────────────┴─────────┴───────┘


C:\Users\magnu\AppData\Local\Temp\ipykernel_32272\4233841252.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)


We see that we need to combine these duplicate rows. We therefore propose that for multiple instances of the same article and user, we combine the readtime and select the largest scroll percentage. This way we can preserve the data without having duplicates

In [17]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.2)
print(train_df.head())

shape: (5, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9786821    ┆ 2292950 ┆ 3740.0         ┆ 100.0      │
│ 9774187    ┆ 1995822 ┆ 229.0          ┆ 100.0      │
│ 9778192    ┆ 2149447 ┆ 62.0           ┆ 100.0      │
│ 9771242    ┆ 1367028 ┆ 63.0           ┆ 100.0      │
│ 9772284    ┆ 1478164 ┆ 36.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [18]:
recommender = UserBasedCollaborativeRecommender(train_df)
recommender.fit(scroll_weight=0.1,readtime_weight=1)

{2292950: [(2250383, np.float64(0.9999886141755959)),
  (346583, np.float64(0.3747026310215875)),
  (2135038, np.float64(0.30661999258385797)),
  (17548, np.float64(0.23733605725132323)),
  (1051979, np.float64(0.15272715608317433)),
  (823326, np.float64(0.13643942566861889)),
  (261979, np.float64(0.11166237154107261)),
  (2236166, np.float64(0.09363852995576116)),
  (2513094, np.float64(0.09295131173461635)),
  (2196889, np.float64(0.05809254145382037))],
 1995822: [(1298524, np.float64(0.9963293752815301)),
  (25262, np.float64(0.99632937528153)),
  (2164418, np.float64(0.99632937528153)),
  (16011, np.float64(0.99632937528153)),
  (2450508, np.float64(0.99632937528153)),
  (385957, np.float64(0.9963293751317129)),
  (1569800, np.float64(0.9963267229426865)),
  (1638545, np.float64(0.9963159619595481)),
  (1802661, np.float64(0.9962952799733952)),
  (924521, np.float64(0.9961340616809411))],
 2149447: [(527085, np.float64(0.9976497385871409)),
  (2290809, np.float64(0.9809386851604

This model treats all interactions equally, no matter how long an interaction lasted

In [19]:
binary_recommender = UserBasedCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{2292950: [(28989, np.float64(0.3202563076101743)),
  (2304679, np.float64(0.2773500981126146)),
  (1824596, np.float64(0.2773500981126146)),
  (1517036, np.float64(0.2773500981126146)),
  (262361, np.float64(0.2773500981126146)),
  (1482170, np.float64(0.2773500981126146)),
  (1664433, np.float64(0.2773500981126146)),
  (1127322, np.float64(0.2773500981126146)),
  (1281639, np.float64(0.2773500981126146)),
  (1574880, np.float64(0.2773500981126146))],
 1995822: [(1397493, np.float64(0.35355339059327373)),
  (415908, np.float64(0.35355339059327373)),
  (2050499, np.float64(0.33333333333333326)),
  (381725, np.float64(0.33333333333333326)),
  (2369007, np.float64(0.33333333333333326)),
  (1573439, np.float64(0.33333333333333326)),
  (1733561, np.float64(0.33333333333333326)),
  (407215, np.float64(0.2672612419124244)),
  (1666245, np.float64(0.2357022603955159)),
  (486656, np.float64(0.2357022603955159))],
 2149447: [(2392037, np.float64(0.20557374623546976)),
  (2286406, np.float64(0.

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model Presentation

### Article Recommendation

In [20]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9778351, 9771168, 9774404, 9790559, 9776497]
reccomended for user  620796 :  [9771224, 9789997, 9759891, 9790756, 9767665]
reccomended for user  1067393 :  [9771113, 9774789, 9785986, 9787230, 9785424]
reccomended for user  1726258 :  [9771224, 9789997, 9759891, 9790756, 9767665]
reccomended for user  17205 :  [9771125, 9779748, 9784702, 9775256, 9775562]


In [21]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9771916, 9779777, 9783509, 9778351, 9775985]
reccomended for user  620796 :  [9783278, 9783334, 9774079, 9769432, 9774392]
reccomended for user  1067393 :  [9787098, 9771113, 9775621, 9786860, 9778731]
reccomended for user  1726258 :  [9771224, 9771242, 9771438, 9786359, 9773744]
reccomended for user  17205 :  [9777804, 9775562, 9771846, 9772925, 9770051]


### Evaluation Scores

#### Without the Ability to Recommend Read Articles

The complex model only reccomending articles the user has not yet read

In [22]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0015454545454545456),
 'NDCG@K': np.float64(0.01100070134463512)}

The binary reccomender model only reccomending articles the user has not yet read

In [23]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0012), 'NDCG@K': np.float64(0.01403414071823345)}

#### With the Ability to Recommend Previously Read Articles

The complex model reccomending articles the user, even if they have read them before

In [24]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.0020833333333333333),
 'NDCG@K': np.float64(0.021821699746875357)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [25]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.001320754716981132),
 'NDCG@K': np.float64(0.011590501922435603)}

## Model Experimentation

In [26]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, 9782092, 9780815, None, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9778351, 9771168, 9774404, 9790559, 9776497, 9776147, 9778945, 9781878, 9783057, 9771333, 9772813, 9772300, 9767830, 9782884, 9782869, 9774120, 9772903, 9781998, 9783657, 9774972, 9774229, 9782423, 9783993, 9774430, 9765759, 9773846, 9786378, 9771330, 9779269, 9778139, 9790784, 9777406, 9780284, 9779408, 9777621, 9772343, 9769917, 9771253, 9780677, 9784425, 9780384, 9789647, 9771919, 9781785, 9776560, 9790987, 9772963, 9775562, 9779737, 9769624, 9776985, 9779724, 9778915, 9780406, 9775881, 9772227, 9780960, 9772475, 9465878, 9773574, 9771916, 9779045, 9771350, 9784696, 9784575, 9784879, 9783655, 9773137, 97

In [27]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9773711, 9774864, 9784591, 9776916, 9775518, 9777464, 9771460, 9787465, 9788362, 9759955, 9776855, 9758424, 9790942, 9771627, 9786351, 9787510, 9771127, 9776246, 9786111}
[9778351, 9771168, 9774404, 9790559, 9776497, 9776147, 9778945, 9781878, 9783057, 9771333, 9772813, 9772300, 9767830, 9782884, 9782869, 9774120, 9772903, 9781998, 9783657, 9774972, 9774229, 9783993, 9782423, 9774430, 9765759, 9773846, 9786378, 9778139, 9771330, 9779269, 9790784, 9777406, 9780284, 9779408, 9777621, 9772343, 9769917, 9771253, 9780677, 9784425, 9780384, 9789647, 9771919, 9781785, 9776560, 9790987, 9772963, 9775562, 9779737, 9779724, 9778915, 9769624, 9776985, 9780406, 9775881, 9772227, 9780960, 9772475, 9465878, 9773574, 9771916, 9779045, 9771350, 9784696, 9784879, 9784575, 9783655, 9773137, 9773210, 9784852, 9779294, 9784710, 9771187, 9777299, 9772442, 9774187, 9780496, 9770327, 9781389, 9784444, 9760091, 9781423]


### Model Evaluation
Performs and appends (to csv file `/evaluation_summary/model_overview.csv`) the model evaluation of precision@k, recall@k and fpr@k.

In [28]:
from utils.evaluation import perform_model_evaluation

matrics = perform_model_evaluation(recommender, test_df, k=5)
matrics

{'precision@k': np.float64(0.00387243735763098),
 'recall@k': np.float64(0.005897947252680182),
 'fpr@k': np.float64(0.0019309245292843408)}

In [29]:
from utils.evaluation import append_model_metrics

append_model_metrics(matrics, "user based CF")

### Diversity Evaluation
Calculates the aggrigate diversity of the recommender model recommendations, and appends the result to the `/evaluation_summary/model_overview_diversity.csv`-file. 

In [30]:
users_ids = train_history_df['user_id'].unique()

In [31]:
from utils.evaluation import aggregate_diversity
from utils.evaluation import append_aggregate_diversity

diversity = aggregate_diversity(recommender, articles_df, users_df=users_ids, user_sample=1000)

print("Diversity")
print(diversity)

append_aggregate_diversity(diversity, "user_based_cf")

Diversity
0.04744912720609509


### Gini

In [34]:
from utils.evaluation import gini_coefficient
from utils.evaluation import append_gini_coefficient
gini = gini_coefficient(recommender, articles_ids_df=articles_df, users_ids_df=users_ids, user_sample=1000)
print("Gini Coefficient")
print(gini)
append_gini_coefficient(gini, "user_based_cf")

Sampling users
Computing Gini coefficient
[9788921, 9783667, 9779092, 9788107, 9789404, 9790784, 9769504, 9779071, 9771473, 9769624, 9786139, 9773282, 9784793, 9772038, 9781756, 9789743, 9786359, 9789703, 9774574, 9786693, 9779653, 9782993, 9789096, 9790942, 9787230, 9782311, 9789911, 9778943, 9767697, 9759891, 9771330, 9778804, 9787846, 9788225, 9755800, 9784662, 9787931, 9777397, 8860119, 9784949, 9771168, 9783790, 9789866, 9772485, 9775785, 9778328, 9778168, 9789065, 9784793, 9788404, 9774032, 9783655, 9781354, 9774789, 9771888, 9781476, 9776322, 9783990, 9790121, 9785475, 9775752, 9772178, 9786293, 9780280, 9775647, 9788766, 9784138, 9784160, 9785209, 9778943, 9785048, 9746810, 9775207, 9787465, 9779648, 9774079, 9771333, 9771919, 9776560, 9770288, 9784951, 9782869, 9780302, 9781476, 9786821, 9788736, 9789910, 9778302, 9776917, 9788106, 9785350, 9790025, 9777565, 9788125, 9789718, 9789473, 9781476, 9777565, 9772475, 9767697, 9789494, 9779538, 9766225, 9789125, 9782092, 9786932, 977

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [ ]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "user_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 12:14:16] [setup] RAM Tracking...
[codecarbon INFO @ 12:14:16] [setup] CPU Tracking...
[codecarbon WARNING @ 12:14:16] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 12:14:17] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 12:14:17] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 12:14:17] [setup] GPU Tracking...
[codecarbon INFO @ 12:14:17] No GPU found.
[codecarbon INFO @ 12:14:17] >>> Tracker's metadata:
[codecarbon INFO @ 12:14:17]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 12:14:17]   Python version: 3.11.9
[codecarbon INFO @ 12:14:17]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 12:14:17]   Available RAM : 15.731 GB
[codecarbon INFO @ 12:14:17]   CPU count: 20
[codecarbon INFO @ 12:14:17]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 12:14:17]   GPU count: None
[codecarbon INFO @ 12:14:17]   GPU model: None
[codecarbon INFO @ 12:14:21] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\user_based_fit_

{'fit': ({1957222: [(934536, np.float64(0.9426891017067498)),
    (439129, np.float64(0.9147931928874817)),
    (2325549, np.float64(0.8690605169475955)),
    (262657, np.float64(0.6993805218030159)),
    (727268, np.float64(0.6615611763316869)),
    (1915689, np.float64(0.5347437343315455)),
    (66603, np.float64(0.5321020729321025)),
    (2523201, np.float64(0.49744498331481823)),
    (945074, np.float64(0.4815410732897871)),
    (881196, np.float64(0.448629653167123))],
   163652: [(1686572, np.float64(0.6068931561026079)),
    (1553127, np.float64(0.6068931561026079)),
    (1637884, np.float64(0.6068931561026079)),
    (1208893, np.float64(0.6068931561026079)),
    (594038, np.float64(0.6068931561026079)),
    (1470089, np.float64(0.6068931561026079)),
    (198059, np.float64(0.6068931561026079)),
    (482294, np.float64(0.6068931561026079)),
    (2347079, np.float64(0.6068931561026079)),
    (889788, np.float64(0.6068931561026079))],
   1734991: [(143807, np.float64(0.30450588970